In [1]:
import ipywidgets as wg
from general_app_layout import AppLayout, LinearLayout
from conc_bm import ConcreteBeam
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
class ConcBeamLayout(object):
    def __init__(self, model=None):
        self.model = model
        self.long_reinf = []
        self.initialize()
    
    def initialize(self):
        
        self.build_input()

        # Initialize calc.
        self.xsect_shape.run_calc()
        self.material.run_calc()
        for b in self.long_reinf:
            b.run_calc()
        self.model.run_calc()

        self.plot_output = wg.Output()
        self.plot()
        
    def build_input(self):
        run_calc_btn = wg.Button(description='Run Calc', layout=wg.Layout(width='auto'))
        run_calc_btn.on_click(self.run_calc)
        
        self.xsect_shape = self.new_xsect_shape()
        self.material = self.new_material()
        
        # Concrete inputs box.
        conc_input = wg.VBox([wg.Label(value='Section Geometry'), self.xsect_shape.b, self.xsect_shape.h, 
                              wg.Label(value='Material Properties'), self.material.fc_prime, self.material.strain_ult, self.material.Ec])

        # Create longitudinal steel new layer button and table.
        new_long_reinf_btn = wg.Button(description='New Bar Layer')
        new_long_reinf_btn.on_click(self.new_long_reinf)
        long_reinf_tbl = self.long_reinf_tbl()
        steel_input = wg.VBox([new_long_reinf_btn, long_reinf_tbl])
        
        # Build the T-beam input layout.
        conc_input_box = wg.Accordion(children=[conc_input])
        conc_input_box.set_title(0, 'Concrete')
        steel_input_box = wg.Accordion(children=[steel_input])
        steel_input_box.set_title(0, 'Steel Reinforcement')
        # Consolidate all input into one box.
        self.input_box = wg.VBox([run_calc_btn, conc_input_box, steel_input_box])
    
        # Create one set of bars.
        self.new_long_reinf(None)
        
    def run_calc(self, b):
        self.xsect_shape.run_calc()
        self.material.run_calc()
        for b in self.long_reinf:
            b.run_calc()
        self.model.run_calc()
        
        # Replot
        self.plot()
        
    def plot(self):
        # Clear existing plots.
        self.plot_output.clear_output()
        # matplotlib
        with self.plot_output:
            fig1, fig2, fig3, fig4 = self.model.plot()
        
    def long_reinf_tbl(self):
        # Define table layout properties.
        col_layout = wg.Layout(width='100%')
        
        # Create longitudinal steel table column labels.
        self.size_lbl = wg.Label(value='size')
        self.qty_lbl = wg.Label(value='qty')
        self.x_lbl = wg.Label(value='x')
        self.y_lbl = wg.Label(value='y')
        self.fy_lbl =  wg.Label(value='fy')
        self.Es_lbl =  wg.Label(value='Es')
        self.delete_lbl = wg.Label(value='Delete')
        
        # Create longitudinal steel columns.
        self.size_box = wg.VBox([self.size_lbl], layout=col_layout)
        self.qty_box = wg.VBox([self.qty_lbl], layout=col_layout)
        self.x_box = wg.VBox([self.x_lbl], layout=col_layout)
        self.y_box = wg.VBox([self.y_lbl], layout=col_layout)
        self.fy_box =  wg.VBox([self.fy_lbl], layout=col_layout)
        self.Es_box =  wg.VBox([self.Es_lbl], layout=col_layout)
        self.delete_box = wg.VBox([self.delete_lbl], layout=col_layout)
        
        # Create longitudinal table.
        long_reinf_tbl = wg.HBox([self.size_box, self.qty_box, self.x_box, self.y_box, self.fy_box, self.Es_box, self.delete_box])
        return long_reinf_tbl

    def new_xsect_shape(self):
        model = self.model.new_xsect_shape()
        layout = XsectShapeLayout(self, model)
        return layout
    
    def new_material(self):
        model = self.model.new_material()
        layout = ConcMaterialLayout(self, model)
        return layout
    
    def new_long_reinf(self, b):
        # Create the new bar layer model and layout. Give the model to the new bar layer layout and add the layout to the 
        # concrete beam layout. 
        model = self.model.new_long_reinf()
        layout = LongReinfLayout(self, model)
        self.long_reinf.append(layout)
        self.rebuild_long_reinf_tbl()

    def delete_long_reinf(self, long_reinf):
        # Remove the bar from the list of bars in both the model and layout and rebuild the bar layer table.
        self.model.long_reinf.remove(long_reinf.model)
        self.long_reinf.remove(long_reinf)
        self.rebuild_long_reinf_tbl()

    def rebuild_long_reinf_tbl(self):
        # Distribute bar info into columns.
        self.size_box.children = tuple([self.size_lbl] + [b.size for b in self.long_reinf])
        self.qty_box.children = tuple([self.qty_lbl] + [b.qty for b in self.long_reinf])
        self.x_box.children = tuple([self.x_lbl] + [b.x for b in self.long_reinf])
        self.y_box.children = tuple([self.y_lbl] + [b.y for b in self.long_reinf])
        self.fy_box.children = tuple([self.fy_lbl] + [b.fy for b in self.long_reinf])
        self.Es_box.children = tuple([self.Es_lbl] + [b.Es for b in self.long_reinf])
        self.delete_box.children = tuple([self.delete_lbl] + [b.delete_btn for b in self.long_reinf])


In [3]:
class ConcMaterialLayout(object):
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')

        self.fc_prime = wg.FloatText(value=model.fc_prime, description='f\'c', layout=cell_layout)
        self.strain_ult = wg.FloatText(value=model.strain_ult, description='strain_ult', layout=cell_layout)
        self.Ec = wg.FloatText(value=model.Ec, description='Ec', layout=cell_layout)
    
    def run_calc(self):
        # Update model.
        self.model.fc_prime = self.fc_prime.value
        self.model.strain_ult = self.strain_ult.value
        self.model.Ec = self.Ec.value
        self.model.run_calc()
        

In [4]:
class XsectShapeLayout(object):
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')

        self.b = wg.FloatText(value=model.b, description='b', layout=cell_layout)
        self.h = wg.FloatText(value=model.h, description='h', layout=cell_layout)
    
    def run_calc(self):
        # Update model.
#         self.model.b = self.b.value
#         self.model.h = self.h.value
        # Run model calc.
        self.model.run_calc(b=self.b.value, h=self.h.value)
        

In [5]:
class LongReinfLayout(object):
    def __init__(self, parent, model):
        self.parent = parent
        self.model = model
        
        cell_layout = wg.Layout(width='auto')
        
        self.size = wg.Text(value=model.size, layout=cell_layout)
        self.qty = wg.FloatText(value=model.qty, layout=cell_layout)
        self.x = wg.FloatText(value=model.x, layout=cell_layout)
        self.y = wg.FloatText(value=model.y, layout=cell_layout)
        self.fy = wg.FloatText(value=model.fy, layout=cell_layout)
        self.Es = wg.FloatText(value=model.Es, layout=cell_layout)
        
        self.delete_btn = wg.Button(description='Delete', layout=cell_layout)
        self.delete_btn.on_click(self.delete)
        
    def delete(self, b):
        # Delete the model and remove the layout.
        # del self.model
        self.parent.delete_long_reinf(self)
        
    def run_calc(self):
        # Update model values.
        self.model.size = self.size.value
        self.model.qty = self.qty.value
        self.model.x = self.x.value
        self.model.y = self.y.value
        self.model.fy = self.fy.value
        self.model.Es = self.Es.value
        # Run model calc.
        self.model.run_calc()
        


In [6]:
beam = ConcBeamLayout(ConcreteBeam())
# app_layout = AppLayout(calc_layout.input_box, calc_layout.plot_box, wg.Box())
app_layout = LinearLayout(beam.input_box, beam.plot_output, wg.Box())
app_layout.app_layout